In [1]:
import math
import json
import pandas as pd
from fuzzywuzzy import fuzz


In [2]:

# Read the GeoJSON file
with open('nat_gas_pipelines.geojson', 'r') as f:
    data = json.load(f)

# Access the features in the GeoJSON file
gas_features = data['features']

with open('crude_oil_pipelines.geojson', 'r') as f:
    data = json.load(f)
    
oil_features = data['features']


In [3]:
for feature in oil_features:
    if 'Opername' in feature['properties']:
        feature['properties']['Operator'] = feature['properties'].pop('Opername')

In [4]:
features = gas_features + oil_features

In [5]:
def calculate_total_distance(features):
    """
    Calculates the total distance of all LineString and MultiLineString geometries in the given list of feature dictionaries.
    
    Args:
        features (list): A list of feature dictionaries, each with a 'geometry' key containing a LineString or MultiLineString geometry.
    
    Returns:
        float: The total distance of all LineString and MultiLineString geometries.
    """
    total_distance = 0
    
    for feature in features:
        geometry = feature.get('geometry', None)
        if geometry is None:
            continue
        
        geometry_type = geometry['type']
        coordinates = geometry['coordinates']
        
        if geometry_type == 'LineString':
            for i in range(len(coordinates) - 1):
                x1, y1 = coordinates[i]
                x2, y2 = coordinates[i+1]
                total_distance += math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        elif geometry_type == 'MultiLineString':
            for linestring in coordinates:
                for i in range(len(linestring) - 1):
                    x1, y1 = linestring[i]
                    x2, y2 = linestring[i+1]
                    total_distance += math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    return total_distance

In [6]:
operators = {}

for x in features:
    
    distance = calculate_total_distance([x])
    x['properties']['Distance'] = distance
    
    operator = x['properties']['Operator'].lower()
    
    
    
    
    if operator in operators:
        operators[operator] = operators[operator] + distance
    else:
        operators[operator] = distance
    

In [12]:
operators = dict(sorted(operators.items()))
operators

{'acadian gas gathering system': 176803.96989088127,
 'acadian gas pipeline sys': 779991.4653032126,
 'acadian haynesville pipeline': 420246.51732460444,
 'agua blanca pipeline': 293457.5269052594,
 'algonquin gas trans co': 1484989.3875583948,
 'alliance pipeline system': 2191554.4498700704,
 'alpine transportation': 146767.15921392798,
 'alyeska pipeline': 2890455.5393740647,
 'amoco pipeline co': 681045.2258404586,
 'anr pipeline co': 14760059.120665118,
 'anr storage co': 44357.83858625556,
 'arkansas western llc': 1080388.0268663904,
 'atmos energy (united cities gas co)': 15817.48516889234,
 'atmos pipeline - texas': 10169733.543215264,
 'birdsboro pipeline': 27422.670585787506,
 'bison pipeline': 672233.6746152296,
 'black marlin pipeline co': 222597.567206034,
 'blue dolphin pipeline co': 173534.5002374365,
 'bluewater pipeline co': 74125.09506172959,
 'bp pipeline (alaska)': 396413.0841004106,
 'bp pipeline (north america)': 1398038.373508439,
 'bridgeline holdings pipeline lp

In [39]:
df = pd.read_csv('oil_spill_data.csv')
df['Operator Name'] = df['Operator Name'].str.lower()
df_sub = df[['Operator Name', 'Accident Year', 'Net Loss (Barrels)']]
df_sub.head(2)

,Operator Name,Accident Year,Net Loss (Barrels)
0,oneok ngl pipeline lp,2010,21.0
1,portland pipeline corp,2010,0.0


In [43]:
df_grouped = df_sub.groupby(['Operator Name','Accident Year']).sum()
df_grouped

Net Loss (Barrels)
Operator Name               Accident Year                    
alyeska pipeline service co 2010                       1119.0
                            2011                          0.0
                            2012                          0.0
                            2013                          0.0
                            2015                          0.0
...                                                       ...
wolverine pipeline co       2014                          9.0
                            2016                          0.0
wyoming pipeline company    2014                          8.5
                            2015                         38.0
                            2016                         11.5

[633 rows x 1 columns]

In [44]:
# Convert the DataFrame to a dictionary, with 'Name' as the key
data_dict = {}
for name in df['Operator Name'].unique():
    name_group = df_grouped.loc[name]
    data_dict[name] = name_group.to_dict('index')
data_dict

{'oneok ngl pipeline lp': {2010: {'Net Loss (Barrels)': 18623.04},
  2011: {'Net Loss (Barrels)': 964.0799999999999},
  2012: {'Net Loss (Barrels)': 24.72}},
 'portland pipeline corp': {2010: {'Net Loss (Barrels)': 0.0}},
 'petrologistics olefins, llc': {2010: {'Net Loss (Barrels)': 3.31},
  2011: {'Net Loss (Barrels)': 5.0},
  2013: {'Net Loss (Barrels)': 14.0},
  2014: {'Net Loss (Barrels)': 7.0}},
 'enbridge energy, limited partnership': {2010: {'Net Loss (Barrels)': 4075.0},
  2011: {'Net Loss (Barrels)': 0.0},
  2012: {'Net Loss (Barrels)': 1874.0},
  2013: {'Net Loss (Barrels)': 0.48},
  2014: {'Net Loss (Barrels)': 0.0},
  2015: {'Net Loss (Barrels)': 0.0},
  2016: {'Net Loss (Barrels)': 0.0}},
 'plains pipeline, l.p.': {2010: {'Net Loss (Barrels)': 2.0},
  2011: {'Net Loss (Barrels)': 0.0},
  2012: {'Net Loss (Barrels)': 15.0},
  2013: {'Net Loss (Barrels)': 0.0},
  2014: {'Net Loss (Barrels)': 40.0},
  2015: {'Net Loss (Barrels)': 1957.0},
  2016: {'Net Loss (Barrels)': 250.0}

In [45]:
# Convert the dictionary to a JSON file
with open('spills_per_year.json', 'w') as json_file:
    json.dump(data_dict, json_file, indent=4)

In [37]:
spill_operators = list(df['Operator Name'].unique())
spill_operators.sort()
spill_operators

['alyeska pipeline service co',
 'amoco oil co',
 'asig - honolulu',
 'belle fourche pipeline co',
 'bhp billiton petroleum (eagle ford gathering) llc',
 'bkep crude, llc',
 'bkep pipeline, llc',
 'blue racer midstream, llc',
 'bp oil pipeline co',
 'bp pipeline (north america) inc.',
 'bp pipelines (alaska), inc',
 'bp usfo/logistics',
 'bp west coast products l.l.c.',
 'bridger lake, llc',
 'bridger pipeline llc',
 'buckeye development & logistics, llc',
 'buckeye gulf coast pipeline lp',
 'buckeye partners, lp',
 'butte pipeline co',
 'calnev pipeline co',
 'calumet montana refining, llc',
 'camino real gathering co llc',
 'ccps transportation, llc',
 'cenex pipeline llc',
 'central florida pipeline corp',
 'centurion pipeline l.p.',
 'chaparral energy, llc',
 'chemoil terminals corp.',
 'chevron midstream pipelines llc',
 'chevron petrochemical pipeline llc',
 'chevron pipe line co',
 'chevron products company',
 'chevron products company - hawaii',
 'chevron u.s.a. inc',
 'citgo p

In [11]:
for index, row in df.iterrows():
    
    name = row['Operator Name']
    if name in operators:
        print(name + ' in operators')

In [13]:
features

[{'type': 'Feature',
  'id': 1,
  'geometry': {'type': 'LineString',
   'coordinates': [[-10530933.3674226, 3460100.31313749],
    [-10530817.5951522, 3459954.78573747]]},
  'properties': {'FID': 1,
   'TYPEPIPE': 'Intrastate',
   'Operator': 'Texas Intrastate Pipeline Co',
   'Status': 'Operating',
   'Shape_Leng': 0.0015401610306,
   'Distance': 185.96086349063492}},
 {'type': 'Feature',
  'id': 2,
  'geometry': {'type': 'LineString',
   'coordinates': [[-10531562.9904625, 3460056.75722927],
    [-10530933.3674226, 3460100.31313749]]},
  'properties': {'FID': 2,
   'TYPEPIPE': 'Intrastate',
   'Operator': 'Texas Intrastate Pipeline Co',
   'Status': 'Operating',
   'Shape_Leng': 0.0056662100208,
   'Distance': 631.127791746932}},
 {'type': 'Feature',
  'id': 3,
  'geometry': {'type': 'LineString',
   'coordinates': [[-10528750.6148471, 3452060.33583791],
    [-10526707.6795521, 3446043.49015274]]},
  'properties': {'FID': 3,
   'TYPEPIPE': 'Intrastate',
   'Operator': 'Texas Intrasta

In [25]:
# Iterate through the list and dictionary
for name_from_list in spill_operators:
    for name_from_dict in operators:
        similarity_ratio = fuzz.ratio(name_from_list, name_from_dict)
        print(f"Similarity ratio between '{name_from_list}' and '{name_from_dict}': {similarity_ratio}%")

Similarity ratio between 'alyeska pipeline service co' and 'acadian gas gathering system': 25%
Similarity ratio between 'alyeska pipeline service co' and 'acadian gas pipeline sys': 51%
Similarity ratio between 'alyeska pipeline service co' and 'acadian haynesville pipeline': 47%
Similarity ratio between 'alyeska pipeline service co' and 'agua blanca pipeline': 51%
Similarity ratio between 'alyeska pipeline service co' and 'algonquin gas trans co': 41%
Similarity ratio between 'alyeska pipeline service co' and 'alliance pipeline system': 59%
Similarity ratio between 'alyeska pipeline service co' and 'alpine transportation': 46%
Similarity ratio between 'alyeska pipeline service co' and 'alyeska pipeline': 74%
Similarity ratio between 'alyeska pipeline service co' and 'amoco pipeline co': 59%
Similarity ratio between 'alyeska pipeline service co' and 'anr pipeline co': 62%
Similarity ratio between 'alyeska pipeline service co' and 'anr storage co': 44%
Similarity ratio between 'alyeska 

Similarity ratio between 'calnev pipeline co' and 'san diego gas & elec co': 49%
Similarity ratio between 'calnev pipeline co' and 'sea robin pipeline': 61%
Similarity ratio between 'calnev pipeline co' and 'shell pipeline company': 65%
Similarity ratio between 'calnev pipeline co' and 'shoshone pipeline': 63%
Similarity ratio between 'calnev pipeline co' and 'sierrita gas pipeline': 51%
Similarity ratio between 'calnev pipeline co' and 'sonat exploration co': 32%
Similarity ratio between 'calnev pipeline co' and 'south carolina pl corp': 40%
Similarity ratio between 'calnev pipeline co' and 'south georgia gas pipeline co': 55%
Similarity ratio between 'calnev pipeline co' and 'southeast supply header pipeline co': 57%
Similarity ratio between 'calnev pipeline co' and 'southern california gas co': 36%
Similarity ratio between 'calnev pipeline co' and 'southern natural gas co': 29%
Similarity ratio between 'calnev pipeline co' and 'southern pines storage lateral': 33%
Similarity ratio b

Similarity ratio between 'consolidated edison co of ny' and 'atmos energy (united cities gas co)': 38%
Similarity ratio between 'consolidated edison co of ny' and 'atmos pipeline - texas': 28%
Similarity ratio between 'consolidated edison co of ny' and 'birdsboro pipeline': 22%
Similarity ratio between 'consolidated edison co of ny' and 'bison pipeline': 29%
Similarity ratio between 'consolidated edison co of ny' and 'black marlin pipeline co': 31%
Similarity ratio between 'consolidated edison co of ny' and 'blue dolphin pipeline co': 31%
Similarity ratio between 'consolidated edison co of ny' and 'bluewater pipeline co': 45%
Similarity ratio between 'consolidated edison co of ny' and 'bp pipeline (alaska)': 17%
Similarity ratio between 'consolidated edison co of ny' and 'bp pipeline (north america)': 22%
Similarity ratio between 'consolidated edison co of ny' and 'bridgeline holdings pipeline lp': 27%
Similarity ratio between 'consolidated edison co of ny' and 'bridger pipeline': 14%


Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'north baja pipeline co': 47%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'north coast gas transmission co': 25%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'north country pipeline co': 45%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'north penn gas co': 24%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'northern border pl co': 41%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'northern illinois gas co': 32%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'northern natural gas co': 25%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'northern utilities (me)': 36%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'northwest pipeline': 39%
Similarity ratio between 'enbridge pipelines (ozark) l.l.c.' and 'northwestern energy co': 22%
Similarity ratio between 'enbridge pipelines

Similarity ratio between 'freeport-mcmoran oil & gas' and 'north baja pipeline co': 21%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'north coast gas transmission co': 35%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'north country pipeline co': 35%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'north penn gas co': 37%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'northern border pl co': 34%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'northern illinois gas co': 48%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'northern natural gas co': 41%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'northern utilities (me)': 37%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'northwest pipeline': 18%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'northwestern energy co': 29%
Similarity ratio between 'freeport-mcmoran oil & gas' and 'nustar energy': 26%
Similarity ratio between 'freeport-

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:

str1 = "portland pipeline"
str2 = "portland pipeline corp"
similarity_ratio = fuzz.ratio(str1, str2)
print(f"The similarity ratio between '{str1}' and '{str2}' is {similarity_ratio}%.")

The similarity ratio between 'portland pipeline' and 'portland pipeline corp' is 87%.
